In [ ]:
import sys
!{sys.executable} -m pip install scipy

In [ ]:
import os
import time
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from numpy.linalg import norm
from scipy.sparse.linalg import svds  # For truncated SVD
from memory_profiler import memory_usage

# --- File paths ---
base_path = r'G:\My Drive\NUS\NUS Y6S1\ME5311\PROJECT_2420_ME5311'
slp_path = os.path.join(base_path, 'slp.nc')
t2m_path = os.path.join(base_path, 't2m.nc')

# --- Load datasets ---
ds_slp = xr.open_dataset(slp_path)
ds_t2m = xr.open_dataset(t2m_path)

slp = ds_slp['msl'].values
t2m = ds_t2m['t2m'].values
timestamps = ds_slp['time'].values
lats = ds_slp['latitude'].values
longs = ds_slp['longitude'].values

# --- Reshape and center SLP ---
n_time, n_lat, n_lon = slp.shape
A_slp = slp.reshape(n_time, -1).T
A_mean_slp = A_slp.mean(axis=1, keepdims=True)
A_centered_slp = A_slp - A_mean_slp

In [ ]:
# --- Define number of singular values/vectors to compute ---
k = 500  # Number of singular values to compute, adjust based on your needs

# --- Define Truncated SVD function with monitoring ---
def perform_truncated_svd_with_monitoring(A, k):
    """Performs Truncated SVD with runtime and memory tracking"""
    print(f"Performing Truncated SVD on SLP data (k={k})...")
    
    def truncated_svd_task():
        global U_slp, S_slp, VT_slp
        
        # Use scipy's svds for truncated SVD
        # Note: svds returns singular values in ascending order, so we need to reverse them
        U_slp, S_slp, VT_slp = svds(A, k=k)
        
        # Reverse the results to get descending order of singular values
        U_slp = U_slp[:, ::-1]
        S_slp = S_slp[::-1]
        VT_slp = VT_slp[::-1]

    # Monitor runtime and memory usage
    start = time.time()
    mem_usage = memory_usage(truncated_svd_task, max_usage=True)
    elapsed = time.time() - start
    
    return elapsed, mem_usage

# --- Run Truncated SVD with performance monitoring ---
elapsed_slp, peak_mem_slp = perform_truncated_svd_with_monitoring(A_centered_slp, k)

# --- SVD result shapes (informational) ---
print(f"A shape: {A_slp.shape}")
print(f"U shape: {U_slp.shape}, S shape: {S_slp.shape}, VT shape: {VT_slp.shape}")
print(f"Truncated to k={k} singular values/vectors")

In [ ]:
# --- Accuracy (Reconstruction Error) ---
def calculate_reconstruction_error(U, S, VT, A_original, A_mean):
    """Calculate reconstruction error using Frobenius norm"""
    # Create diagonal S matrix for matrix multiplication
    S_diag = np.diag(S)
    
    # Reconstruct the original matrix
    A_reconstructed = U @ S_diag @ VT + A_mean
    
    # Calculate relative error
    error = norm(A_original - A_reconstructed) / norm(A_original)
    return error, A_reconstructed

# --- Noise Robustness Test ---
def test_noise_robustness(A_centered, A_original, A_mean, k, noise_scale=0.01):
    """Test Truncated SVD robustness against Gaussian noise"""
    np.random.seed(0)  # For reproducibility
    noise = np.random.normal(scale=noise_scale, size=A_centered.shape)
    A_noisy = A_centered + noise
    
    # Perform Truncated SVD on noisy data
    U_noisy, S_noisy, VT_noisy = svds(A_noisy, k=k)
    
    # Reverse for descending order
    U_noisy = U_noisy[:, ::-1]
    S_noisy = S_noisy[::-1]
    VT_noisy = VT_noisy[::-1]
    
    # Calculate reconstruction error with noise
    S_noisy_diag = np.diag(S_noisy)
    A_reconstructed_noisy = U_noisy @ S_noisy_diag @ VT_noisy + A_mean
    
    error = norm(A_original - A_reconstructed_noisy) / norm(A_original)
    return error

# Calculate reconstruction error
reconstruction_error, A_reconstructed = calculate_reconstruction_error(
    U_slp, S_slp, VT_slp, A_slp, A_mean_slp
)

# Test noise robustness
noise_error = test_noise_robustness(A_centered_slp, A_slp, A_mean_slp, k)

# --- Report results ---
print("\n===== Truncated SVD Results for SLP =====")
print(f"Number of singular values/vectors: k = {k}")
print(f"Runtime: {elapsed_slp:.2f} seconds")
print(f"Peak memory usage: {peak_mem_slp:.2f} MiB")
print(f"Reconstruction error (Frobenius norm): {reconstruction_error:.6e}")
print(f"Noise robustness (error with Gaussian noise): {noise_error:.6e}")

# --- Cumulative energy ---
total_energy = np.sum(S_slp**2)
cumulative_energy = np.cumsum(S_slp**2) / total_energy
energy_90 = np.where(cumulative_energy >= 0.9)[0][0] + 1
energy_95 = np.where(cumulative_energy >= 0.95)[0][0] + 1

print(f"Number of modes for 90% energy: {energy_90}")
print(f"Number of modes for 95% energy: {energy_95}")

# --- Optional: Save results to file for later comparison ---
results = {
    "method": "Truncated SVD",
    "k_value": k,
    "runtime": elapsed_slp,
    "memory_usage": peak_mem_slp,
    "reconstruction_error": float(reconstruction_error),
    "noise_robustness": float(noise_error),
    "modes_90pct_energy": int(energy_90),
    "modes_95pct_energy": int(energy_95),
    "top_singular_values": S_slp[:10].tolist()  # Save first 10 singular values
}

# Save as JSON (optional)
import json
with open("truncated_svd_results.json", "w") as f:
    json.dump(results, f, indent=4)

# --- Plot singular value decay ---
plt.figure(figsize=(10, 6))
plt.semilogy(range(1, len(S_slp) + 1), S_slp, 'o-')
plt.title(f'Singular Value Decay (k={k}) - Truncated SVD')
plt.xlabel('Index')
plt.ylabel('Singular Value (log scale)')
plt.grid(True)
plt.savefig('truncated_svd_singular_values.png', dpi=300)
plt.show()

# --- Plot cumulative energy ---
plt.figure(figsize=(10, 6))
plt.plot(range(1, len(S_slp) + 1), cumulative_energy, 'o-')
plt.axhline(y=0.9, color='r', linestyle='--', label='90% Energy')
plt.axhline(y=0.95, color='g', linestyle='--', label='95% Energy')
plt.title('Cumulative Energy vs. Number of Modes')
plt.xlabel('Number of Modes')
plt.ylabel('Cumulative Energy Fraction')
plt.legend()
plt.grid(True)
plt.savefig('truncated_svd_cumulative_energy.png', dpi=300)
plt.show()